***
# 08. 순환 신경망(Recurrent Neural Network)
***

앞서 배운 피드 포워드 신경망은 입력의 길이가 고정되어 있어 자연어 처리를 위한 신경망으로는 한계가 있었습니다. 결국 다양한 길이의 입력 시퀀스를 처리할 수 있는 인공 신경망이 필요하게 되었는데, 자연어 처리에 대표적으로 사용되는 인공 신경망이 바로 순환 신경망(Recurrent Neural Network, RNN)입니다. 이번 챕터에서는 가장 기본적인 순환 신경망인 바닐라 RNN, 이를 개선한 LSTM, GRU에 대해서 학습해봅시다. LSTM과 GRU를 이해한다면 텍스트 분류나 기계 번역과 같은 다양한 자연어 처리 문제들을 풀 수 있습니다.

## 08-01 순환 신경망(Recurrent Neural Network, RNN)
***
RNN(Recurrent Neural Network)은 입력과 출력을 시퀀스 단위로 처리하는 시퀀스(Sequence) 모델입니다. 번역기를 생각해보면 입력은 번역하고자 하는 단어의 시퀀스인 문장입니다. 출력에 해당되는 번역된 문장 또한 단어의 시퀀스입니다. 이와 같이 시퀀스들을 처리하기 위해 고안된 모델들을 시퀀스 모델이라고 합니다. 그 중 RNN은 가장 기본적인 인공 신경망 시퀀스 모델입니다.

뒤에서 배우는 LSTM이나 GRU 또한 근본적으로 RNN에 속합니다. RNN을 이해하고 'RNN을 이용한 텍스트 분류' 챕터, '태깅 작업' 챕터, 'RNN을 이용한 인코더-디코더' 챕터에서 실습을 진행합니다.

용어는 비슷하지만 순환 신경망과 재귀 신경망(Recursive Neural Network)은 전혀 다른 개념입니다.  

### 1. 순환 신경망(Recurrent Neural Network, RNN)
***

앞서 배운 신경망들은 전부 은닉층에서 활성화 함수를 지난 값은 오직 출력층 방향으로만 향했습니다. 이와 같은 신경망들을 피드 포워드 신경망(Feed Forward Neural Network)이라고 합니다. 그런데 그렇지 않은 신경망들이 있습니다. RNN(Recurrent Neural Network) 또한 그 중 하나입니다. RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고있습니다.

![그림](img.png)

위의 그림을 보겠습니다. $x$는 입력층의 입력 벡터, $y$는 출력층의 출력 벡터입니다.
실제로는 편향 $b$도 입력으로 존재할 수 있지만 앞으로의 그림에서는 생략합니다.
RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드를 셀(cell)이라고 합니다.
이 셀은 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행하므로 이를 메모리 셀 또는 RNN 셀이라고 표현합니다.

은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 하고 있습니다.
앞으로는 현재 시점을 변수 t로 표현하겠습니다. 이는 현재 시점 t에서의 메모리 셀이 갖고있는 값은 과거의 메모리 셀들의 값에 영향을 받은 것임을 의미합니다.
그렇다면 메모리 셀이 갖고 있는 이 값은 뭐라고 부를까요?

메모리 셀이 출력층 방향 또는 다음 시점인 t+1의 자신에게 보내는 값을 은닉 상태(hidden state) 라고 합니다.
다시 말해 t 시점의 메모리 셀은 t-1 시점의 메모리 셀이 보낸 은닉 상태값을 t 시점의 은닉 상태 계산을 위한 입력값으로 사용합니다.

![그림](img_1.png)

RNN을 표현할 때는 일반적으로 위의 그림에서 좌측과 같이 화살표로 사이클을 그려서 재귀 형태로 표현하기도 하지만,
우측과 같이 사이클을 그리는 화살표 대신 여러 시점으로 펼쳐서 표현하기도 합니다. 두 그림은 동일한 그림으로 단지 사이클을 그리는 화살표를 사용하여 표현하였느냐,
시점의 흐름에 따라서 표현하였느냐의 차이일 뿐 둘 다 동일한 RNN을 표현하고 있습니다.

피드 포워드 신경망에서는 뉴런이라는 단위를 사용했지만, RNN에서는 뉴런이라는 단위보다는 입력층과 출력층에서는 각각 입력 벡터와 출력 벡터,
은닉층에서는 은닉 상태라는 표현을 주로 사용합니다. 위의 그림에서 회색과 초록색으로 표현한 각 네모들은 기본적으로 벡터 단위를 가정하고 있습니다.
피드 포워드 신경망과의 차이를 비교하기 위해서 RNN을 뉴런 단위로 시각화해보겠습니다.

![그림](img_2.png)

위의 그림은 입력 벡터의 차원이 4, 은닉 상태의 크기가 2, 출력층의 출력 벡터의 차원이 2인 RNN이 시점이 2일 때의 모습을 보여줍니다.
다시 말해 뉴런 단위로 해석하면 입력층의 뉴런 수는 4, 은닉층의 뉴런 수는 2, 출력층의 뉴런 수는 2입니다.

![그림](img_3.png)

RNN은 입력과 출력의 길이를 다르게 설계 할 수 있으므로 다양한 용도로 사용할 수 있습니다. 위 그림은 입력과 출력의 길이에 따라서 달라지는 RNN의 다양한 형태를 보여줍니다. 위 구조가 자연어 처리에서 어떻게 사용될 수 있는지 예를 들어봅시다. RNN 셀의 각 시점의 입, 출력의 단위는 사용자가 정의하기 나름이지만 가장 보편적인 단위는 '단어 벡터'입니다.

예를 들어 하나의 입력에 대해서 여러개의 출력을 의미하는 일 대 다(one-to-many) 구조의 모델은 하나의 이미지 입력에 대해서 사진의 제목을 출력하는 이미지 캡셔닝(Image Captioning) 작업에 사용할 수 있습니다. 사진의 제목은 단어들의 나열이므로 시퀀스 출력입니다.

![그림](img_4.png)

또한 단어 시퀀스에 대해서 하나의 출력을 하는 다 대 일(many-to-one) 구조의 모델은 입력 문서가 긍정적인지 부정적인지를 판별하는 감성 분류(sentiment classification),
또는 메일이 정상 메일인지 스팸 메일인지 판별하는 스팸 메일 분류(spam detection) 등에 사용할 수 있습니다. 위 그림은 RNN으로 스팸 메일을 분류할 때의 아키텍처를 보여줍니다.
이러한 예제들은 'RNN을 이용한 텍스트 분류' 챕터에서 배웁니다.

![그림](img_5.png)

다 대 다(many-to-many) 구조의 모델의 경우에는 사용자가 문장을 입력하면 대답 문장을 출력하는 챗봇과 입력 문장으로부터 번역된 문장을 출력하는 번역기,
또는 '태깅 작업' 챕터에서 배우는 개체명 인식이나 품사 태깅과 같은 작업이 속합니다. 위 그림은 개체명 인식을 수행할 때의 RNN 아키텍처를 보여줍니다.

RNN에 대한 수식을 정의해보겠습니다.

![그림](img_6.png)

현재 시점 t에서의 은닉 상태값을 
$ht$라고 정의하겠습니다. 은닉층의 메모리 셀은 
$ht$를 계산하기 위해서 총 두 개의 가중치를 가집니다. 하나는 입력층을 위한 가중치 
$Wx$이고, 하나는 이전 시점 t-1의 은닉 상태값인 
$ht-1$을 위한 가중치 
$Wh$입니다.

이를 식으로 표현하면 다음과 같습니다.

![그림](img_7.png)

RNN의 은닉층 연산을 벡터와 행렬 연산으로 이해해봅시다. 자연어 처리에서 RNN의 입력 
$xt$는 대부분의 경우 단어 벡터로 간주할 수 있는데, 단어 벡터의 차원을 
$d$라고 하고, 은닉 상태의 크기를 
$Dh$라고 하였을 때 각 벡터와 행렬의 크기는 다음과 같습니다.

![그림](img_8.png)

배치 크기가 1이고, 
$d$와 $Dh$두 값 모두를 4로 가정하였을 때, RNN의 은닉층 연산을 그림으로 표현하면 아래와 같습니다.

![그림](img_9.png)

이때 $ht$
를 계산하기 위한 활성화 함수로는 주로 하이퍼볼릭탄젠트 함수(tanh)가 사용됩니다.
위의 식에서 각각의 가중치 $Wx$, $Wh$, $Wy$의 값은 하나의 층에서는 모든 시점에서 값을 동일하게 공유합니다.
하지만 은닉층이 2개 이상일 경우에는 각 은닉층에서의 가중치는 서로 다릅니다.

출력층은 결과값인 $yt$를 계산하기 위한 활성화 함수로는 푸는 문제에 따라서 다를텐데,
예를 들어서 이진 분류를 해야하는 경우라면 출력층에 로지스틱 회귀를 사용하여 시그모이드 함수를 사용할 수 있고
다중 클래스 분류를 해야하는 경우라면 출력층에 소프트맥스 회귀를 사용하여로 소프트맥스 함수를 사용할 수 있습니다.

### 2. 케라스(Keras)로 RNN 구현하기
***
케라스로 RNN 층을 추가하는 코드는 다음과 같습니다.

In [2]:
from tensorflow.keras.models import Sequential

hidden_units = 32  # 예시로 32로 설정
timesteps = 10  # 예시로 10으로 설정
input_dim = 8  # 예시로 8로 설정
M = 10
N = 8

model = Sequential()

In [3]:
from tensorflow.keras.layers import SimpleRNN

model.add(SimpleRNN(hidden_units))

인자를 사용할 때를 보겠습니다.

In [8]:
# 추가 인자를 사용할 때
model.add(SimpleRNN(hidden_units, input_shape=(timesteps, input_dim)))

# 다른 표기
model.add(SimpleRNN(hidden_units, input_length=M, input_dim=N))

- **hidden_units** = 은닉 상태의 크기를 정의. 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)와도 동일. 
RNN의 용량(capacity)을 늘린다고 보면 되며, 중소형 모델의 경우 보통 128, 256, 512, 1024 등의 값을 가진다.
- **timesteps** = 입력 시퀀스의 길이(input_length)라고 표현하기도 함. 시점의 수.
- **input_dim** = 입력의 크기.

![그림](img_10.png)

RNN 층은 (batch_size, timesteps, input_dim) 크기의 3D 텐서를 입력으로 받습니다.
batch_size는 한 번에 학습하는 데이터의 개수를 말합니다. 여기서부터는 텐서의 개념을 반드시 이해해야 하므로 벡터와 행렬 연산 챕터의 텐서 설명을 참고하시기 바랍니다.
다만, 이러한 표현은 사람이나 문헌에 따라서, 또는 풀고자 하는 문제에 따라서 종종 다르게 기재되는데 의 그림은 문제와 상황에 따라서 다르게 표현되는 입력 3D 텐서의 대표적인 표현들을 보여줍니다.

주의할 점은 위 코드는 출력층까지 포함한 인공 신경망 코드가 아니라 주로 은닉층으로 간주할 수 있는 하나의 RNN 층에 대한 코드입니다.
해당 코드가 리턴하는 결과값은 하나의 은닉 상태 또는 정의하기에 따라 여러 개의 시점의 은닉 상태 입니다.
아래의 그림은 만약 전결합층(Fully-connected layer)을 출력층으로 사용하였을 경우의 인공 신경망 그림과 은닉층까지만 표현한 그림의 차이를 보여줍니다.

![그림](img_11.png)

그렇다면 RNN 층은 위에서 설명한 입력 3D 텐서를 입력받아서 어떻게 은닉 상태를 출력할까요? RNN 층은 사용자의 설정에 따라 두 가지 종류의 출력을 내보냅니다.
메모리 셀의 최종 시점의 은닉 상태만을 리턴하고자 한다면 (batch_size, output_dim) 크기의 2D 텐서를 리턴합니다.
하지만, 메모리 셀의 각 시점(time step)의 은닉 상태값들을 모아서 전체 시퀀스를 리턴하고자 한다면 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴합니다.
이는 RNN 층의 return_sequences 매개 변수에 True를 설정하여 설정이 가능합니다. output_dim은 앞서 코드에서 정의한 hidden_units의 값으로 설정됩니다.

![그림](img_12.png)

위의 그림은 time step=3일 때, return_sequences = True를 설정했을 때와 그렇지 않았을 때 어떤 차이가 있는지를 보여줍니다.
return_sequences=True를 선택하면 메모리 셀이 모든 시점(time step)에 대해서 은닉 상태값을 출력하며,
별도 기재하지 않거나 return_sequences=False로 선택할 경우에는 메모리 셀은 하나의 은닉 상태값만을 출력합니다.
그리고 이 하나의 값은 마지막 시점(time step)의 메모리 셀의 은닉 상태값입니다.

마지막 은닉 상태만 전달하도록 하면 다 대 일(many-to-one) 문제를 풀 수 있고, 모든 시점의 은닉 상태를 전달하도록 하면,
다음층에 RNN 은닉층이 하나 더 있는 경우이거나 다 대 다(many-to-many) 문제를 풀 수 있습니다.

뒤에서 배우는 LSTM이나 GRU도 내부 메커니즘은 다르지만 model.add()를 통해서 층을 추가하는 코드는 SimpleRNN 코드와 같은 형태를 가집니다.
실습을 통해 모델 내부적으로 출력 결과를 어떻게 정의하는지 이해해봅시다.

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, 3)                 42        
                                                                 
Total params: 42 (168.00 Byte)
Trainable params: 42 (168.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


출력값이 (batch_size, output_dim) 크기의 2D 텐서일 때,
output_dim은 hidden_units의 값인 3입니다. 이 경우 batch_size를 현 단계에서는 알 수 없으므로 (None, 3)이 됩니다.
이번에는 batch_size를 미리 정의해보겠습니다.

In [10]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_7 (SimpleRNN)    (8, 3)                    42        
                                                                 
Total params: 42 (168.00 Byte)
Trainable params: 42 (168.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


batch_size를 8로 기재하면 출력의 크기가 (8, 3)이 됩니다.
return_sequences 매개 변수에 True를 기재하여 출력값으로 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴하도록 모델을 만들어 보겠습니다.

In [11]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_8 (SimpleRNN)    (8, 2, 3)                 42        
                                                                 
Total params: 42 (168.00 Byte)
Trainable params: 42 (168.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


출력의 크기가 (8, 2, 3)이 됩니다.

### 3. 파이썬으로 RNN 구현하기
***

직접 Numpy로 RNN 층을 구현해보겠습니다. 앞서 메모리 셀에서 은닉 상태를 계산하는 식을 다음과 같이 정의하였습니다.

![그림](img_13.png)

실제 구현에 앞서 간단히 가상의 코드(pseudocode)를 작성해보겠습니다.

```
# 아래의 코드는 가상의 코드(pseudocode)로 실제 동작하는 코드가 아님. 

hidden_state_t = 0 # 초기 은닉 상태를 0(벡터)로 초기화
for input_t in input_length: # 각 시점마다 입력을 받는다.
    output_t = tanh(input_t, hidden_state_t) # 각 시점에 대해서 입력과 은닉 상태를 가지고 연산
    hidden_state_t = output_t # 계산 결과는 현재 시점의 은닉 상태가 된다.
```

우선 t 시점의 은닉 상태를 hidden_state_t라는 변수로 선언하였고, 입력 데이터의 길이를 input_length로 선언하였습니다.
이 경우, 입력 데이터의 길이는 곧 총 시점의 수(timesteps)가 됩니다. 그리고 t 시점의 입력값을 input_t로 선언하였습니다.
각 메모리 셀은 각 시점마다 input_t와 hidden_sate_t(이전 상태의 은닉 상태)를 입력으로 활성화 함수인 하이퍼볼릭탄젠트 함수를 통해 현 시점의 hidden_state_t를 계산합니다.

가상의 코드를 통해 간단히 개념 정립을 해보았습니다. RNN 층을 실제 동작되는 코드로 구현해보겠습니다.
아래의 코드는 이해를 돕기 위해 (timesteps, input_dim) 크기의 2D 텐서를 입력으로 받았다고 가정하였으나,
실제로 케라스에서는 (batch_size, timesteps, input_dim)의 크기의 3D 텐서를 입력으로 받는 것을 기억합시다.

timesteps는 시점의 수입니다. 자연어 처리에서는 보통 문장의 길이입니다.
input_dim은 입력의 차원입니다. 자연어 처리에서는 보통 단어 벡터의 차원입니다. hidden_units는 은닉 상태의 크기로 메모리 셀의 용량입니다.
초기 은닉 상태는 0의 값을 가지는 벡터로 초기화합니다. 초기 은닉 상태를 출력해보겠습니다.

In [12]:
import numpy as np

timesteps = 10
input_dim = 4
hidden_units = 8

# 입력에 해당되는 2D 텐서
inputs = np.random.random((timesteps, input_dim))

# 초기 은닉 상태는 0(벡터)로 초기화
hidden_state_t = np.zeros((hidden_units,)) 

print('초기 은닉 상태 :',hidden_state_t)


초기 은닉 상태 : [0. 0. 0. 0. 0. 0. 0. 0.]


은닉 상태의 크기를 8로 정의하였으므로 8의 차원을 가지는 0의 값으로 구성된 벡터가 출력됩니다.
가중치와 편향을 각 크기에 맞게 정의하고 크기를 출력해보겠습니다.

In [13]:
Wx = np.random.random((hidden_units, input_dim))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_units, hidden_units)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_units,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

print('가중치 Wx의 크기(shape) :',np.shape(Wx))
print('가중치 Wh의 크기(shape) :',np.shape(Wh))
print('편향의 크기(shape) :',np.shape(b))

가중치 Wx의 크기(shape) : (8, 4)
가중치 Wh의 크기(shape) : (8, 8)
편향의 크기(shape) : (8,)


각 가중치와 편향의 크기는 다음과 같습니다. Wx는 (은닉 상태의 크기 × 입력의 차원), Wh는 (은닉 상태의 크기 × 은닉 상태의 크기), b는 (은닉 상태의 크기)의 크기를 가집니다.
이제 모든 시점의 은닉 상태를 출력한다고 가정하고, RNN 층을 동작시켜봅시다.

In [14]:
total_hidden_states = []

# 각 시점 별 입력값.
for input_t in inputs:

  # Wx * Xt + Wh * Ht-1 + b(bias)
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b)

  # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep t, output_dim)
  # 각 시점의 은닉 상태의 값을 계속해서 누적
  total_hidden_states.append(list(output_t))
  hidden_state_t = output_t

# 출력 시 값을 깔끔하게 해주는 용도.
total_hidden_states = np.stack(total_hidden_states, axis = 0) 

# (timesteps, output_dim)
print('모든 시점의 은닉 상태 :')
print(total_hidden_states)

모든 시점의 은닉 상태 :
[[0.80391446 0.86834143 0.85764737 0.91008318 0.51830353 0.83934155
  0.98091778 0.96534425]
 [0.99984269 0.99974302 0.99976639 0.99882364 0.99967892 0.99877968
  0.99999736 0.99991036]
 [0.99990821 0.99981453 0.99993513 0.99917845 0.99982178 0.99765399
  0.99999766 0.99991854]
 [0.99987955 0.99983257 0.99983218 0.99883418 0.99991018 0.99847675
  0.99999577 0.99988348]
 [0.99986066 0.99978357 0.99983185 0.9989331  0.99990375 0.99893061
  0.99999749 0.99992688]
 [0.99994045 0.99993191 0.99984744 0.99924116 0.99994979 0.99899738
  0.99999679 0.99988017]
 [0.99996905 0.99995712 0.99992395 0.99954034 0.99995178 0.99925306
  0.99999863 0.99993347]
 [0.99996812 0.99991867 0.99996278 0.99953491 0.99992911 0.99951286
  0.99999955 0.9999845 ]
 [0.99997972 0.99996694 0.9999448  0.99959914 0.99996463 0.99956214
  0.99999924 0.99996774]
 [0.99996774 0.99995671 0.99988056 0.99939418 0.99997518 0.99967615
  0.99999882 0.99996316]]


### 4. 깊은 순환 신경망(Deep Recurrent Neural Network)
***

![그림](img_14.png)

앞서 RNN도 다수의 은닉층을 가질 수 있다고 언급한 바 있습니다.
위의 그림은 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 순환 신경망의 모습을 보여줍니다.
은닉층을 2개 추가하는 경우 코드는 아래와 같습니다.

In [15]:
model = Sequential()
model.add(SimpleRNN(hidden_units, input_length=10, input_dim=5, return_sequences=True))
model.add(SimpleRNN(hidden_units, return_sequences=True))

위의 코드에서 첫번째 은닉층은 다음 은닉층이 존재하므로 return_sequences = True를 설정하여 모든 시점에 대해서 은닉 상태 값을 다음 은닉층으로 보내주고 있습니다.

### 5. 양방향 순환 신경망(Bidirectional Recurrent Neural Network)
***

양방향 순환 신경망은 시점 t에서의 출력값을 예측할 때 이전 시점의 입력뿐만 아니라,
이후 시점의 입력 또한 예측에 기여할 수 있다는 아이디어에 기반합니다. 빈칸 채우기 문제에 비유하여 보겠습니다.

운동을 열심히 하는 것은 [        ]을 늘리는데 효과적이다.

1) 근육
2) 지방
3) 스트레스

'운동을 열심히 하는 것은 [ ]을 늘리는데 효과적이다.' 라는 문장에서 문맥 상으로 정답은 '근육'입니다.
위의 빈 칸 채우기 문제를 풀 때 이전에 나온 단어들만으로 빈 칸을 채우려고 시도해보면 정보가 부족합니다.
'운동을 열심히 하는 것은' 까지만 주고 뒤의 단어들은 가린 채 빈 칸의 정답이 될 수 있는 세 개의 선택지 중 고르는 것은 뒤의 단어들까지 알고있는 상태보다 명백히 정답을 결정하기가 어렵습니다.

RNN이 풀고자 하는 문제 중에서는 과거 시점의 입력 뿐만 아니라 미래 시점의 입력에 힌트가 있는 경우도 많습니다.
그래서 이전과 이후의 시점 모두를 고려해서 현재 시점의 예측을 더욱 정확하게 할 수 있도록 고안된 것이 양방향 RNN입니다.

![그림](img_15.png)

양방향 RNN은 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용합니다.
첫번째 메모리 셀은 앞에서 배운 것처럼 앞 시점의 은닉 상태(Forward States) 를 전달받아 현재의 은닉 상태를 계산합니다.
위의 그림에서는 주황색 메모리 셀에 해당됩니다. 두번째 메모리 셀은 앞에서 배운 것과는 다릅니다.
앞 시점의 은닉 상태가 아니라 뒤 시점의 은닉 상태(Backward States) 를 전달 받아 현재의 은닉 상태를 계산합니다.
입력 시퀀스를 반대 방향으로 읽는 것입니다. 위의 그림에서는 초록색 메모리 셀에 해당됩니다. 그리고 이 두 개의 값 모두가 현재 시점의 출력층에서 출력값을 예측하기 위해 사용됩니다.

In [16]:
from tensorflow.keras.layers import Bidirectional

timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))

양방향 RNN도 다수의 은닉층을 가질 수 있습니다.
아래의 그림은 양방향 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 양방향 순환 신경망의 모습을 보여줍니다.

![그림](img_16.png)

다른 인공 신경망 모델들도 마찬가지이지만,
은닉층을 무조건 추가한다고 해서 모델의 성능이 좋아지는 것은 아닙니다. 은닉층을 추가하면 학습할 수 있는 양이 많아지지만 반대로 훈련 데이터 또한 많은 양이 필요합니다.
아래의 코드는 은닉층이 4개인 경우를 보여줍니다.

In [17]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))

양방향 RNN은 태깅 작업 챕터의 실습에서 사용해보겠습니다.

### 6. 점검 퀴즈
***

RNN을 제대로 이해했는지 퀴즈를 통해서 확인해보세요! 모델에 대한 설명이 다음과 같을 때, 총 파라미터 개수를 구해보세요.

1. Embedding을 사용하며, 단어 집합(Vocabulary)의 크기가 5,000이고 임베딩 벡터의 차원은 100입니다.
2. 은닉층에서는 Simple RNN을 사용하며, 은닉 상태의 크기는 128입니다.
3. 훈련에 사용하는 모든 샘플의 길이는 30으로 가정합니다.
4. 이진 분류를 수행하는 모델로, 출력층의 뉴런은 1개로 시그모이드 함수를 사용합니다.
5. 은닉층은 1개입니다.

> **1. Embedding 레이어 파라미터 수: 단어 집합 크기 × 임베딩 벡터의 차원**
  > 5000 × 100 = 500,000

> **2. Simple RNN 레이어 파라미터 수:**
  > 입력에서 은닉층으로의 가중치: 입력 차원 × 은닉 상태의 크기 
  은닉층에서 은닉층으로의 가중치: 은닉 상태의 크기 × 은닉 상태의 크기
  편향(bias): 은닉 상태의 크기
  (입력 차원 = 임베딩 벡터의 차원, 은닉 상태의 크기 = 128)
  입력에서 은닉층으로의 가중치: 100 × 128 = 12,800
  은닉층에서 은닉층으로의 가중치: 128 × 128 = 16,384
  편향: 128

> **3. 출력층 파라미터 수:**
  >입력 차원 × 출력 차원 + 편향
  입력 차원 = 은닉 상태의 크기, 출력 차원 = 1
  128 × 1 + 1 = 129

따라서 총 파라미터 수는 각 레이어의 파라미터 수를 합산한 것입니다.

500,000 (Embedding) + 12,800 (Simple RNN 입력에서 은닉층) + 16,384 (Simple RNN 은닉층에서 은닉층) + 128 (Simple RNN 편향) + 129 (출력층) = 529,441

따라서 총 파라미터 수는 529,441개입니다.

## 08-02 장단기 메모리(Long Short-Term Memory, LSTM)
***

바닐라 아이스크림이 가장 기본적인 맛을 가진 아이스크림인 것처럼, 앞서 배운 RNN을 가장 단순한 형태의 RNN이라고 하여 바닐라 RNN(Vanilla RNN)이라고 합니다.
(케라스에서는 SimpleRNN) 바닐라 RNN 이후 바닐라 RNN의 한계를 극복하기 위한 다양한 RNN의 변형이 나왔습니다.
이번에 배우게 될 LSTM도 그 중 하나입니다. 앞으로의 설명에서 LSTM과 비교하여 RNN을 언급하는 것은 전부 바닐라 RNN을 말합니다.

### 1. 바닐라 RNN의 한계
***

![그림](img_17.png)

앞에서 바닐라 RNN은 출력 결과가 이전의 계산 결과에 의존한다는 것을 언급한 바 있습니다.
하지만 바닐라 RNN은 비교적 짧은 시퀀스(sequence)에 대해서만 효과를 보이는 단점이 있습니다.
바닐라 RNN의 시점(time step)이 길어질 수록 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 발생합니다.
위의 그림은 첫번째 입력값인 $x1$의 정보량을 짙은 남색으로 표현했을 때,
색이 점차 얕아지는 것으로 시점이 지날수록 $x1$의 정보량이 손실되어가는 과정을 표현하였습니다.
뒤로 갈수록 $x1$의 정보량은 손실되고,
시점이 충분히 긴 상황에서는 $x1$의 전체 정보에 대한 영향력은 거의 의미가 없을 수도 있습니다.

어쩌면 가장 중요한 정보가 시점의 앞 쪽에 위치할 수도 있습니다.
RNN으로 만든 언어 모델이 다음 단어를 예측하는 과정을 생각해봅시다.
예를 들어 "모스크바에 여행을 왔는데 건물도 예쁘고 먹을 것도 맛있었어. 그런데 글쎄 직장 상사한테 전화가 왔어. 어디냐고 묻더라구 그래서 나는 말했지. 저 여행왔는데요. 여기 ___"
다음 단어를 예측하기 위해서는 장소 정보가 필요합니다. 그런데 장소 정보에 해당되는 단어인 '모스크바'는 앞에 위치하고 있고, RNN이 충분한 기억력을 가지고 있지 못한다면 다음 단어를 엉뚱하게 예측합니다.

이를 **장기 의존성 문제(the problem of Long-Term Dependencies)** 라고 합니다.

### 2. 바닐라 RNN 내부 열어보기
***

![그림](img_18.png)

LSTM에 대해서 이해해보기 전에 바닐라 RNN의 뚜껑을 열어보겠습니다. 위의 그림은 바닐라 RNN의 내부 구조를 보여줍니다.
이 책에서는 RNN 계열의 인공 신경망의 그림에서는 편향 $b$를 생략합니다.
위의 그림에 편향 $b$를 그린다면 $xt$옆에 tanh로 향하는 또 하나의 입력선을 그리면 됩니다.

![그림](img_19.png)

바닐라 RNN은 $xt$와 $ht-1$이라는 두 개의 입력이 각각의 가중치와 곱해져서 메모리 셀의 입력이 됩니다.
그리고 이를 하이퍼볼릭탄젠트 함수의 입력으로 사용하고 이 값은 은닉층의 출력인 은닉 상태가 됩니다.

### 3. LSTM(Long Short-Term Memory)
***

![그림](img_20.png)

위의 그림은 LSTM의 전체적인 내부의 모습을 보여줍니다. 전통적인 RNN의 이러한 단점을 보완한 RNN의 일종을 장단기 메모리(Long Short-Term Memory)라고 하며, 줄여서 LSTM이라고 합니다.
LSTM은 은닉층의 메모리 셀에 입력 게이트, 망각 게이트, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야할 것들을 정합니다.
요약하면 LSTM은 은닉 상태(hidden state)를 계산하는 식이 전통적인 RNN보다 조금 더 복잡해졌으며 셀 상태(cell state)라는 값을 추가하였습니다.
위의 그림에서는 t시점의 셀 상태를 $Ct$로 표현하고 있습니다. LSTM은 RNN과 비교하여 긴 시퀀스의 입력을 처리하는데 탁월한 성능을 보입니다.

![그림](img_21.png)

셀 상태는 위의 그림에서 왼쪽에서 오른쪽으로 가는 굵은 선입니다. 셀 상태 또한 이전에 배운 은닉 상태처럼 이전 시점의 셀 상태가 다음 시점의 셀 상태를 구하기 위한 입력으로서 사용됩니다.

은닉 상태의 값과 셀 상태의 값을 구하기 위해서 새로 추가 된 3개의 게이트를 사용합니다.
각 게이트는 삭제 게이트, 입력 게이트, 출력 게이트라고 부르며 이 3개의 게이트에는 공통적으로 시그모이드 함수가 존재합니다.
시그모이드 함수를 지나면 0과 1사이의 값이 나오게 되는데 이 값들을 가지고 게이트를 조절합니다. 아래의 내용을 참고로 각 게이트에 대해서 알아보겠습니다.

![그림](img_22.png)

#### (1) 입력 게이트

![그림](img_23.png)

입력 게이트는 현재 정보를 기억하기 위한 게이트입니다.
우선 현재 시점 t의 $x$값과 입력 게이트로 이어지는 가중치 $Wxi$를 곱한 값과 이전 시점 t-1의 은닉 상태가 입력 게이트로 이어지는 가중치 $Whi$를 곱한 값을 더하여 시그모이드 함수를 지납니다.
이를 $it$라고 합니다.
그리고 현재 시점 t의 $x$값과 입력 게이트로 이어지는 가중치 $Wxg$를 곱한 값과 이전 시점 t-1의 은닉 상태가 입력 게이트로 이어지는 가중치 $Whg$를 곱한 값을 더하여 하이퍼볼릭탄젠트 함수를 지납니다.
이를 $gt$라고 합니다.
시그모이드 함수를 지나 0과 1사이의 값을 가지는 $it$과 하이퍼볼릭탄젠트 함수를 지나 -1과 1사이의 값을 가지는 $gt$.
이 두 개의 값을 가지고 이번에 선택된 기억할 정보의 양을 정하는데, 구체적으로 어떻게 결정하는지는 아래에서 배우게 될 셀 상태 수식을 참고합니다.

#### (2) 삭제 게이트

![그림](img_24.png)

삭제 게이트는 기억을 삭제하기 위한 게이트입니다. 현재 시점 t의 $x$값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지나게 됩니다.
시그모이드 함수를 지나면 0과 1 사이의 값이 나오게 되는데, 이 값이 곧 삭제 과정을 거친 정보의 양입니다. 0에 가까울수록 정보가 많이 삭제된 것이고 1에 가까울수록 정보를 온전히 기억한 것입니다.
이를 가지고 셀 상태를 구하게 되는데, 구체적으로는 아래에서 배우게 될 셀 상태 수식을 참고합니다.

#### (3) 셀 상태

![그림](img_25.png)

셀 상태 $Ct$를 구하는 방법을 알아보겠습니다.
삭제 게이트에서 일부 기억을 잃은 상태입니다.

입력 게이트에서 구한 $it$, $gt$이 두 개의 값에 대해서 원소별 곱(entrywise product)을 진행합니다.
다시 말해 같은 크기의 두 행렬이 있을 때 같은 위치의 성분끼리 곱하는 것을 말합니다.
여기서는 식으로 ○로 표현합니다. 이것이 이번에 선택된 기억할 값입니다.

입력 게이트에서 선택된 기억을 삭제 게이트의 결과값과 더합니다.
이 값을 현재 시점 t의 셀 상태라고 하며, 이 값은 다음 t+1 시점의 LSTM 셀로 넘겨집니다.

삭제 게이트와 입력 게이트의 영향력을 이해해봅시다. 만약 삭제 게이트의 출력값인 $ft$가 0이 된다면,
이전 시점의 셀 상태의 값인 $Ct-1$은 현재 시점의 셀 상태의 값을 결정하기 위한 영향력이 0이 되면서,
오직 입력 게이트의 결과만이 현재 시점의 셀 상태의 값 $Ct$을 결정할 수 있습니다.
이는 삭제 게이트가 완전히 닫히고 입력 게이트를 연 상태를 의미합니다. 반대로 입력 게이트의 $it$값을 0이라고 한다면,
현재 시점의 셀 상태의 값 $Ct$는 오직 이전 시점의 셀 상태의 값 $Ct-1$의 값에만 의존합니다.
이는 입력 게이트를 완전히 닫고 삭제 게이트만을 연 상태를 의미합니다.
결과적으로 삭제 게이트는 이전 시점의 입력을 얼마나 반영할지를 의미하고, 입력 게이트는 현재 시점의 입력을 얼마나 반영할지를 결정합니다.

#### (4) 출력 게이트와 은닉 상태

![그림](img_26.png)

출력 게이트는 현재 시점 t의 $x$값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지난 값입니다.
해당 값은 현재 시점 t의 은닉 상태를 결정하는 일에 쓰이게 됩니다.
셀 상태의 값이 하이퍼볼릭탄젠트 함수를 지나 -1과 1사이의 값이 되고, 해당 값은 출력 게이트의 값과 연산되면서,
값이 걸러지는 효과가 발생하여 은닉 상태가 됩니다. 은닉 상태의 값은 또한 출력층으로도 향합니다.

## 08-03 게이트 순환 유닛(Gated Recurrent Unit, GRU)
***

GRU(Gated Recurrent Unit)는 2014년 뉴욕대학교 조경현 교수님이 집필한 논문에서 제안되었습니다.
GRU는 LSTM의 장기 의존성 문제에 대한 해결책을 유지하면서, 은닉 상태를 업데이트하는 계산을 줄였습니다.
다시 말해서, GRU는 성능은 LSTM과 유사하면서 복잡했던 LSTM의 구조를 간단화 시켰습니다.

### 1. GRU(Gated Recurrent Unit)
***
LSTM에서는 출력, 입력, 삭제 게이트라는 3개의 게이트가 존재했습니다. 반면, GRU에서는 업데이트 게이트와 리셋 게이트 두 가지 게이트만이 존재합니다.
GRU는 LSTM보다 학습 속도가 빠르다고 알려져있지만 여러 평가에서 GRU는 LSTM과 비슷한 성능을 보인다고 알려져 있습니다.

![그림](img_27.png)

GRU와 LSTM 중 어떤 것이 모델의 성능면에서 더 낫다라고 단정지어 말할 수 없으며,
기존에 LSTM을 사용하면서 최적의 하이퍼파라미터를 찾아낸 상황이라면 굳이 GRU로 바꿔서 사용할 필요는 없습니다.

경험적으로 데이터 양이 적을 때는 매개 변수의 양이 적은 GRU가 조금 더 낫고,
데이터 양이 더 많으면 LSTM이 더 낫다고도 합니다. GRU보다 LSTM에 대한 연구나 사용량이 더 많은데, 이는 LSTM이 더 먼저 나온 구조이기 때문입니다.

### 2. 케라스에서의 GRU(Gated Recurrent Unit)
***

케라스에서는 역시 GRU에 대한 구현을 지원합니다. 사용 방법은 SimpleRNN이나 LSTM과 같습니다.
```
model.add(GRU(hidden_size, input_shape=(timesteps, input_dim)))
```

## 08-04 케라스의 SimpleRNN과 LSTM 이해하기
***

### 1. 임의의 입력 생성하기
***
우선 RNN과 LSTM을 테스트하기 위한 임의의 입력을 만듭니다.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

train_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
print(np.shape(train_X))

(4, 5)


위 입력은 단어 벡터의 차원은 5이고, 문장의 길이가 4인 경우를 가정한 입력입니다.
다시 말해 4번의 시점(timesteps)이 존재하고, 각 시점마다 5차원의 단어 벡터가 입력으로 사용됩니다.
그런데 앞서 RNN은 2D 텐서가 아니라 3D 텐서를 입력을 받는다고 언급한 바 있습니다. 즉, 위에서 만든 2D 텐서를 3D 텐서로 변경합니다.
이는 배치 크기 1을 추가해주므로서 해결합니다.

In [4]:
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


(batch_size, timesteps, input_dim)에 해당되는 (1, 4, 5)의 크기를 가지는 3D 텐서가 생성되었습니다.
batch_size는 한 번에 RNN이 학습하는 데이터의 양을 의미하지만, 여기서는 샘플이 1개 밖에 없으므로 batch_size는 1입니다.

### 2. SimpleRNN 이해하기
***

위에서 생성한 데이터를 SimpleRNN의 입력으로 사용하여 SimpleRNN의 출력값을 이해해보겠습니다.
SimpleRNN에는 여러 인자가 있으며 대표적인 인자로 return_sequences와 return_state가 있습니다.
기본값으로는 둘 다 False로 지정되어져 있으므로 별도 지정을 하지 않을 경우에는 False로 처리됩니다.
우선, 은닉 상태의 크기를 3으로 지정하고, 두 인자 값이 모두 False일 때의 출력값을 보겠습니다.

앞으로의 실습에서 SimpleRNN을 매번 재선언하므로 은닉 상태의 값 자체는 매번 초기화되어 이전 출력과 값의 일관성은 없습니다.
그래서 출력값 자체보다는 해당 값의 크기(shape)에 주목해야합니다.

In [6]:
rnn = SimpleRNN(3)
# rnn = SimpleRNN(3, return_sequences=False, return_state=False)와 동일.
hidden_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[ 0.2123618  -0.91719645 -0.99081415]], shape: (1, 3)


(1, 3) 크기의 텐서가 출력되는데, 이는 마지막 시점의 은닉 상태입니다. 은닉 상태의 크기를 3으로 지정했음을 주목합시다.
기본적으로 return_sequences가 False인 경우에는 SimpleRNN은 마지막 시점의 은닉 상태만 출력합니다.
이번에는 return_sequences를 True로 지정하여 모든 시점의 은닉 상태를 출력해봅시다.

In [7]:
rnn = SimpleRNN(3, return_sequences=True)
hidden_states = rnn(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))

hidden states : [[[-0.9289805  -0.94042486  0.99999523]
  [-0.18654655 -0.86894464  0.9972785 ]
  [ 0.34180346 -0.9009113   0.99761707]
  [-0.99502325 -0.9972721   0.4474207 ]]], shape: (1, 4, 3)


(1, 4, 3) 크기의 텐서가 출력됩니다. 앞서 입력 데이터는 (1, 4, 5)의 크기를 가지는 3D 텐서였고,
그 중 4가 시점(timesteps)에 해당하는 값이므로 모든 시점에 대해서 은닉 상태의 값을 출력하여 (1, 4, 3) 크기의 텐서를 출력하는 것입니다.

return_state가 True일 경우에는 return_sequences의 True/False 여부와 상관없이 마지막 시점의 은닉 상태를 출력합니다.
가령, return_sequences가 True이면서, return_state를 True로 할 경우 SimpleRNN은 두 개의 출력을 리턴합니다.

In [8]:
rnn = SimpleRNN(3, return_sequences=True, return_state=True)
hidden_states, last_state = rnn(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden states : [[[-0.9712732   0.9961506   0.9999672 ]
  [-0.99963707  0.9990745   0.99887353]
  [-0.99906105  0.9994232   0.82860684]
  [-0.9375129   0.9915658   0.8017729 ]]], shape: (1, 4, 3)
last hidden state : [[-0.9375129  0.9915658  0.8017729]], shape: (1, 3)


첫번째 출력은 return_sequences=True로 인한 출력으로 모든 시점의 은닉 상태입니다.
두번째 출력은 return_state=True로 인한 출력으로 마지막 시점의 은닉 상태입니다.
실제로 출력을 보면 모든 시점의 은닉 상태인 (1, 4, 3) 텐서의 마지막 벡터값이 return_state=True로 인해 출력된 벡터값과 일치하는 것을 볼 수 있습니다.
(둘 다 [-0.5144398 -0.5037417 0.96605766])

그렇다면 return_sequences는 False인데, retun_state가 True인 경우는 어떨까요?

In [9]:
rnn = SimpleRNN(3, return_sequences=False, return_state=True)
hidden_state, last_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden state : [[-0.5602136  -0.74207145  0.50586736]], shape: (1, 3)
last hidden state : [[-0.5602136  -0.74207145  0.50586736]], shape: (1, 3)


두 개의 출력 모두 마지막 시점의 은닉 상태를 출력하게 됩니다.

### 3. LSTM 이해하기
***

실제로 SimpleRNN이 사용되는 경우는 거의 없습니다. 이보다는 LSTM이나 GRU을 주로 사용하는데,
이번에는 임의의 입력에 대해서 LSTM을 사용할 경우를 보겠습니다. 우선 return_sequences를 False로 두고, return_state가 True인 경우를 봅시다.

In [10]:
lstm = LSTM(3, return_sequences=False, return_state=True)
hidden_state, last_state, last_cell_state = lstm(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden state : [[ 0.628958   0.4765781 -0.534618 ]], shape: (1, 3)
last hidden state : [[ 0.628958   0.4765781 -0.534618 ]], shape: (1, 3)
last cell state : [[ 1.1673744  2.0724816 -0.8964503]], shape: (1, 3)


이번에는 SimpleRNN 때와는 달리, 세 개의 결과를 반환합니다.
return_sequences가 False이므로 우선 첫번째 결과는 마지막 시점의 은닉 상태입니다.
그런데 LSTM이 SimpleRNN과 다른 점은 return_state를 True로 둔 경우에는 마지막 시점의 은닉 상태뿐만 아니라 셀 상태까지 반환한다는 점입니다.
이번에는 return_sequences를 True로 바꿔보겠습니다.

In [11]:
lstm = LSTM(3, return_sequences=True, return_state=True)
hidden_states, last_hidden_state, last_cell_state = lstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_hidden_state, last_hidden_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden states : [[[-0.05612729 -0.27200848 -0.3696597 ]
  [ 0.0686404  -0.46068686 -0.62259305]
  [ 0.3325969  -0.43366536 -0.5967156 ]
  [ 0.24502099  0.02448091 -0.40157774]]], shape: (1, 4, 3)
last hidden state : [[ 0.24502099  0.02448091 -0.40157774]], shape: (1, 3)
last cell state : [[ 0.5514432   0.06295171 -1.1583989 ]], shape: (1, 3)


return_state가 True이므로 두번째 출력값이 마지막 은닉 상태,
세번째 출력값이 마지막 셀 상태인 것은 변함없지만 return_sequences가 True이므로 첫번째 출력값은 모든 시점의 은닉 상태가 출력됩니다.

### 4. Bidirectional(LSTM) 이해하기
***

난이도를 조금 올려서 양방향 LSTM의 출력값을 확인해보겠습니다.
return_sequences가 True인 경우와 False인 경우에 대해서 은닉 상태의 값이 어떻게 바뀌는지 직접 비교하기 위해서 이번에는 출력되는 은닉 상태의 값을 고정시켜주겠습니다.

In [12]:
k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

우선 return_sequences가 False이고, return_state가 True인 경우입니다.

In [13]:
bilstm = Bidirectional(LSTM(3, return_sequences=False, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[0.63031393 0.63031393 0.63031393 0.7038734  0.7038734  0.7038734 ]], shape: (1, 6)
forward state : [[0.63031393 0.63031393 0.63031393]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


이번에는 무려 5개의 값을 반환합니다. return_state가 True인 경우에는 정방향 LSTM의 은닉 상태와 셀 상태, 역방향 LSTM의 은닉 상태와 셀 상태 4가지를 반환하기 때문입니다.
다만, 셀 상태는 각각 forward_c와 backward_c에 저장만 하고 출력하지 않았습니다. 첫번째 출력값의 크기가 (1, 6)인 것에 주목합시다.
이는 return_sequences가 False인 경우 정방향 LSTM의 마지막 시점의 은닉 상태와 역방향 LSTM의 첫번째 시점의 은닉 상태가 연결된 채 반환되기 때문입니다.
그림으로 표현하면 아래와 같이 연결되어 다음층에서 사용됩니다.

![그림](img_28.png)

마찬가지로 return_state가 True인 경우에 반환한 은닉 상태의 값인 forward_h와 backward_h는 각각 정방향 LSTM의 마지막 시점의 은닉 상태와 역방향 LSTM의 첫번째 시점의 은닉 상태값입니다.
그리고 이 두 값을 연결한 값이 hidden_states에 출력되는 값입니다.
이를 이용한 실습은 'RNN을 이용한 텍스트 분류 챕터'에서의 한국어 스팀 리뷰 분류하기 실습( https://wikidocs.net/94748 )을 참고하세요.

정방향 LSTM의 마지막 시점의 은닉 상태값과 역방향 LSTM의 첫번째 은닉 상태값을 기억해둡시다.

- 정방향 LSTM의 마지막 시점의 은닉 상태값 : [0.6303139 0.6303139 0.6303139]
- 역방향 LSTM의 첫번째 시점의 은닉 상태값 : [0.70387346 0.70387346 0.70387346]

현재 은닉 상태의 값을 고정시켜두었기 때문에 return_sequences를 True로 할 경우, 출력이 어떻게 바뀌는지 비교가 가능합니다.

In [15]:
bilstm = Bidirectional(LSTM(3, return_sequences=True, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

In [16]:
print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[[0.35906473 0.35906473 0.35906473 0.7038734  0.7038734  0.7038734 ]
  [0.5511133  0.5511133  0.5511133  0.58863586 0.58863586 0.58863586]
  [0.59115756 0.59115756 0.59115756 0.3951699  0.3951699  0.3951699 ]
  [0.63031393 0.63031393 0.63031393 0.21942244 0.21942244 0.21942244]]], shape: (1, 4, 6)
forward state : [[0.63031393 0.63031393 0.63031393]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


hidden states의 출력값에서는 이제 모든 시점의 은닉 상태가 출력됩니다.
역방향 LSTM의 첫번째 시점의 은닉 상태는 더 이상 정방향 LSTM의 마지막 시점의 은닉 상태와 연결되는 것이 아니라 정방향 LSTM의 첫번째 시점의 은닉 상태와 연결됩니다.

그림으로 표현하면 다음과 같이 연결되어 다음층의 입력으로 사용됩니다.

![그림](img_29.png)

## 08-05 RNN 언어 모델(Recurrent Neural Network Language Model, RNNLM)
***

### 1. RNN 언어 모델(Recurrent Neural Network Language Model, RNNLM)
***
앞서 n-gram 언어 모델과 NNLM은 고정된 개수의 단어만을 입력으로 받아야한다는 단점이 있었습니다.
하지만 시점(time step)이라는 개념이 도입된 RNN으로 언어 모델을 만들면 입력의 길이를 고정하지 않을 수 있습니다.
이처럼 RNN으로 만든 언어 모델을 RNNLM(Recurrent Neural Network Language Model)이라고 합니다.

RNNLM이 언어 모델링을 학습하는 과정을 보겠습니다. 이해를 위해 간소화 된 형태로 설명합니다.

예문 : 'what will the fat cat sit on'

예를 들어 훈련 코퍼스에 위와 같은 문장이 있다고 해봅시다.
언어 모델은 주어진 단어 시퀀스로부터 다음 단어를 예측하는 모델입니다.
아래의 그림은 RNNLM이 어떻게 이전 시점의 단어들과 현재 시점의 단어로 다음 단어를 예측하는지를 보여줍니다.

![그림](img_30.png)

RNNLM은 기본적으로 예측 과정에서 이전 시점의 출력을 현재 시점의 입력으로 합니다.
RNNLM은 what을 입력받으면, will을 예측하고 이 will은 다음 시점의 입력이 되어 the를 예측합니다.
그리고 the는 또 다시 다음 시점의 입력이 되고 해당 시점에서는 fat을 예측합니다.
그리고 이 또한 다시 다음 시점의 입력이 됩니다.
결과적으로 세번째 시점에서 fat은 앞서 나온 what, will, the라는 시퀀스로 인해 결정된 단어이며,
네번째 시점의 cat은 앞서 나온 what, will, the, fat이라는 시퀀스로 인해 결정된 단어입니다.

사실 위 과정은 훈련이 끝난 모델의 테스트 과정 동안(실제 사용할 때)의 이야기입니다.
훈련 과정에서는 이전 시점의 예측 결과를 다음 시점의 입력으로 넣으면서 예측하는 것이 아니라,
what will the fat cat sit on라는 훈련 샘플이 있다면,
what will the fat cat sit 시퀀스를 모델의 입력으로 넣으면,
will the fat cat sit on를 예측하도록 훈련됩니다.
will, the, fat, cat, sit, on는 각 시점의 레이블입니다.

이러한 RNN 훈련 기법을 교사 강요(teacher forcing)라고 합니다.
교사 강요(teacher forcing)란, 테스트 과정에서 t 시점의 출력이 t+1 시점의 입력으로 사용되는 RNN 모델을 훈련시킬 때 사용하는 훈련 기법입니다.
훈련할 때 교사 강요를 사용할 경우, 모델이 t 시점에서 예측한 값을 t+1 시점에 입력으로 사용하지 않고,
t 시점의 레이블. 즉, 실제 알고있는 정답을 t+1 시점의 입력으로 사용합니다.

물론, 훈련 과정에서도 이전 시점의 출력을 다음 시점의 입력으로 사용하면서 훈련 시킬 수도 있지만
이는 한 번 잘못 예측하면 뒤에서의 예측까지 영향을 미쳐 훈련 시간이 느려지게 되므로 교사 강요를 사용하여 RNN을 좀 더 빠르고 효과적으로 훈련시킬 수 있습니다.

![그림](img_31.png)

훈련 과정 동안 출력층에서 사용하는 활성화 함수는 소프트맥스 함수입니다.
그리고 모델이 예측한 값과 실제 레이블과의 오차를 계산하기 위해서 손실 함수로 크로스 엔트로피 함수를 사용합니다.
이해를 돕기 위해 앞서 배운 NNLM의 그림과 유사한 형태로 RNNLM을 다시 시각화해보겠습니다.

![그림](img_32.png)

RNNLM의 구조를 보겠습니다. RNNLM은 위의 그림과 같이 총 4개의 층(layer)으로 이루어진 인공 신경망입니다.
우선 입력층(input layer)을 봅시다. RNNLM의 현 시점(timestep)은 4로 가정합니다. 그래서 4번째 입력 단어인 fat의 원-핫 벡터가 입력이 됩니다.

출력층(output layer)을 봅시다. 모델이 예측해야하는 정답에 해당되는 단어 cat의 원-핫 벡터는 출력층에서 모델이 예측한 값의 오차를 구하기 위해 사용될 예정입니다.
그리고 이 오차로부터 손실 함수를 사용해 인공 신경망이 학습을 하게 됩니다. 조금 더 구체적으로 살펴보겠습니다.

![그림](img_33.png)

현 시점의 입력 단어의 원-핫 벡터 $xt$를 입력 받은 RNNLM은 우선 임베딩층(embedding layer)을 지납니다.
이 임베딩층은 기본적으로 NNLM에서 배운 투사층(projection layer)입니다.
NNLM에서는 룩업 테이블을 수행하는 층을 투사층라고 표현했지만,
이미 투사층의 결과로 얻는 벡터를 임베딩 벡터라고 부른다고 NNLM에서 학습하였으므로,
앞으로는 임베딩 벡터를 얻는 투사층을 임베딩층(embedding layer)이라는 표현을 사용할 겁니다.

단어 집합의 크기가 V일 때, 임베딩 벡터의 크기를 M으로 설정하면, 각 입력 단어들은 임베딩층에서 V × M 크기의 임베딩 행렬과 곱해집니다.
여기서 V는 단어 집합의 크기를 의미합니다. 만약 원-핫 벡터의 차원이 7이고, M이 5라면 임베딩 행렬은 7 × 5 행렬이 됩니다.
그리고 이 임베딩 행렬은 역전파 과정에서 다른 가중치들과 함께 학습됩니다. 이는 NNLM에서 이미 배운 개념입니다.

**임베딩층 :**

![그림](img_34.png)

***

이 임베딩 벡터는 은닉층에서 이전 시점의 은닉 상태인 $ht-1$과 함께 다음의 연산을 하여 현재 시점의 은닉 상태 
$ht$를 계산하게 됩니다.

**은닉층 :**

![그림](img_35.png)

***

출력층에서는 활성화 함수로 소프트맥스(softmax) 함수를 사용하는데,
V차원의 벡터는 소프트맥스 함수를 지나면서 각 원소는 0과 1사이의 실수값을 가지며 총 합은 1이 되는 상태로 바뀝니다.
이렇게 나온 벡터를 RNNLM의 t시점의 예측값이라는 의미에서 $\hat{y}t$라고 합시다. 이를 식으로 표현하면 아래와 같습니다.

**출력층 :** 

![그림](img_36.png)

벡터 $\hat{y}t$의 각 차원 안에서의 값이 의미하는 것은 이와 같습니다. 
$\hat{y}t$의 j번째 인덱스가 가진 0과 1사이의 값은 j번째 단어가 다음 단어일 확률을 나타냅니다.
그리고 $\hat{y}t$는 실제값. 즉, 실제 정답에 해당되는 단어인 원-핫 벡터의 값에 가까워져야 합니다.
실제값에 해당되는 다음 단어를 $y$라고 했을 때, 이 두 벡터가 가까워지게 하기위해서 RNNLM는 손실 함수로 cross-entropy 함수를 사용합니다.
그리고 역전파가 이루어지면서 가중치 행렬들이 학습되는데, 이 과정에서 임베딩 벡터값들도 학습이 됩니다.

룩업 테이블의 대상이 되는 테이블인 임베딩 행렬을 $E$라고 하였을 때,
결과적으로 RNNLM에서 학습 과정에서 학습되는 가중치 행렬은 다음의 $E$, $Wx$, $Wh$, $Wy$ 4개 입니다.
뒤의 글자 단위 RNN 실습에서 RNN 언어 모델을 구현해보면서 훈련 과정과 테스트 과정의 차이를 이해해보겠습니다.

## 08-06 RNN을 이용한 텍스트 생성(Text Generation using RNN)
***

다 대 일(many-to-one) 구조의 RNN을 사용하여 문맥을 반영해서 텍스트를 생성하는 모델을 만들어봅시다.

### 1. RNN을 이용하여 텍스트 생성하기
***
예를 들어서 '경마장에 있는 말이 뛰고 있다'와 '그의 말이 법이다'와 '가는 말이 고와야 오는 말이 곱다'라는 세 가지 문장이 있다고 해봅시다. 모델이 문맥을 학습할 수 있도록 전체 문장의 앞의 단어들을 전부 고려하여 학습하도록 데이터를 재구성한다면 아래와 같이 총 11개의 샘플이 구성됩니다.

![표](img_37.png)

#### 1) 데이터에 대한 이해와 전처리

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

우선 예제로 언급한 3개의 한국어 문장을 저장합니다.

In [3]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

단어 집합을 생성하고 크기를 확인해보겠습니다.
단어 집합의 크기를 저장할 때는 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
패딩을 위한 0을 고려하여 +1을 해줍니다.

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 고려하여 +1 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


각 단어와 단어에 부여된 정수 인덱스를 출력해보겠습니다.

In [7]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


훈련 데이터를 만들어보겠습니다.

In [8]:
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


샘플의 개수는 총 11개가 나옵니다. 전체 샘플을 출력해봅시다.

In [9]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터입니다.
[2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당됩니다.
전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 합니다.

우선 전체 샘플에 대해서 길이를 일치시켜 줍니다. 가장 긴 샘플의 길이를 기준으로 합니다.
현재 육안으로 봤을 때, 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6입니다.
이를 코드로는 다음과 같이 구할 수 있습니다.

In [10]:
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


전체 훈련 데이터에서 가장 긴 샘플의 길이가 6임을 확인하였습니다. 전체 샘플의 길이를 6으로 패딩합니다.

In [11]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

pad_sequences()는 모든 샘플에 대해서 0을 사용하여 길이를 맞춰줍니다.
maxlen의 값으로 6을 주면 모든 샘플의 길이를 6으로 맞춰주며, padding의 인자로 'pre'를 주면 길이가 6보다 짧은 샘플의 앞에 0으로 채웁니다.
전체 훈련 데이터를 출력해봅니다.

In [12]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


길이가 6보다 짧은 모든 샘플에 대해서 앞에 0을 채워서 모든 샘플의 길이를 6으로 바꿨습니다.
이제 각 샘플의 마지막 단어를 레이블로 분리합시다. 레이블의 분리는 Numpy를 이용해서 가능합니다.
리스트의 마지막 값을 제외하고 저장한 것은 X, 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됩니다.

In [13]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

분리된 X와 y에 대해서 출력해보면 다음과 같습니다.

In [15]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [16]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


레이블이 분리되었습니다. RNN 모델에 훈련 데이터를 훈련 시키기 전에 레이블에 대해서 원-핫 인코딩을 수행합니다.

In [17]:
y = to_categorical(y, num_classes=vocab_size)

원-핫 인코딩이 수행되었는지 출력합니다.

In [18]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


정상적으로 원-핫 인코딩이 수행된 것을 볼 수 있습니다.

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] # 3에 대한 원-핫 벡터
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] # 1에 대한 원-핫 벡터
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.] # 4에 대한 원-핫 벡터
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] # 5에 대한 원-핫 벡터
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] # 1에 대한 원-핫 벡터
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] # 7에 대한 원-핫 벡터
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] # 1에 대한 원-핫 벡터
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] # 9에 대한 원-핫 벡터
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] # 10에 대한 원-핫 벡터
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] # 1에 대한 원-핫 벡터
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]] # 11에 대한 원-핫 벡터

#### 2) 모델 설계하기
RNN 모델에 데이터를 훈련시킵니다.

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

하이퍼파라미터인 임베딩 벡터의 차원은 10, 은닉 상태의 크기는 32입니다. 다 대 일 구조의 RNN을 사용합니다.
전결합층(Fully Connected Layer)을 출력층으로 단어 집합 크기만큼의 뉴런을 배치하여 모델을 설계합니다.
해당 모델은 마지막 시점에서 모든 가능한 단어 중 하나의 단어를 예측하는 다중 클래스 분류 문제를 수행하는 모델입니다.
다중 클래스 분류 문제의 경우, 출력층에 소프트맥스 회귀를 사용해야 하므로 활성화 함수로는 소프트맥스 함수를 사용하고,
손실 함수로 크로스 엔트로피 함수를 사용하여 200 에포크를 수행합니다.

In [20]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4766 - accuracy: 0.0909 - 1s/epoch - 1s/step
Epoch 2/200
1/1 - 0s - loss: 2.4627 - accuracy: 0.0909 - 4ms/epoch - 4ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4487 - accuracy: 0.2727 - 3ms/epoch - 3ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4347 - accuracy: 0.3636 - 4ms/epoch - 4ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4204 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4058 - accuracy: 0.4545 - 3ms/epoch - 3ms/step
Epoch 7/200
1/1 - 0s - loss: 2.3908 - accuracy: 0.4545 - 4ms/epoch - 4ms/step
Epoch 8/200
1/1 - 0s - loss: 2.3754 - accuracy: 0.4545 - 4ms/epoch - 4ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3593 - accuracy: 0.4545 - 3ms/epoch - 3ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3426 - accuracy: 0.4545 - 3ms/epoch - 3ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3253 - accuracy: 0.4545 - 3ms/epoch - 3ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3071 - accuracy: 0.4545 - 4ms/epoch - 4ms/step
Epoch 13/200
1/1 - 0s - loss: 2.2881 - accuracy: 0.4545 - 3ms/e

모델이 정확하게 예측하고 있는지 문장을 생성하는 함수를 만들어서 출력해봅시다.

In [21]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

입력된 단어로부터 다음 단어를 예측해서 문장을 생성하는 함수를 만들었습니다. '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측해봅시다.

In [22]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [23]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [24]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다


앞의 문맥을 기준으로 '말이' 라는 단어 다음에 나올 단어를 기존의 훈련 데이터와 일치하게 예측함을 보여줍니다.
이 모델은 충분한 훈련 데이터를 갖고 있지 못하므로 위에서 문장의 길이에 맞게 적절하게 예측해야하는 횟수 4, 2, 5를 각각 인자값으로 주었습니다.
이 이상의 숫자를 주면 기계는 '있다', '법이다', '곱다' 다음에 나오는 단어가 무엇인지 배운 적이 없으므로 임의 예측을 합니다.
이번에는 더 많은 훈련 데이터를 가지고 실습해봅시다.

### 2. LSTM을 이용하여 텍스트 생성하기
***

사용할 데이터는 뉴욕 타임즈 기사의 제목입니다. 아래의 링크에서 ArticlesApril2018.csv 데이터를 다운로드 합니다.

파일 다운로드 링크 : https://www.kaggle.com/aashita/nyt-comments

In [26]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

다운로드한 훈련 데이터를 데이터프레임에 저장합니다.

In [27]:
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


열의 개수가 굉장히 많기 때문에 한 눈에 보기 어렵습니다. 어떤 열이 있고, 열이 총 몇 개가 있는지 출력해봅시다.

In [28]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


총 15개의 열이 존재합니다.
여기서 사용할 열은 제목에 해당되는 headline 열입니다. Null 값이 있는지 확인해봅시다.

In [29]:
print(df['headline'].isnull().values.any())

False


Null 값은 별도로 없는 것으로 보입니다. headline 열에서 모든 신문 기사의 제목을 뽑아서 하나의 리스트로 저장해보도록 하겠습니다.

In [30]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df.headline.values)) 
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

headline이라는 리스트에 모든 신문 기사의 제목을 저장했습니다. 저장한 리스트에서 상위 5개만 출력해보았습니다.

네번째와 다섯 번째 샘플에 Unknown 값이 들어가있습니다. headline 전체에 걸쳐서 Unknown 값을 가진 샘플이 있을 것으로 추정됩니다.
비록 Null 값은 아니지만 실습에 도움이 되지 않는 노이즈 데이터이므로 제거해줄 필요가 있습니다.
제거하기 전에 현재 샘플의 개수를 확인해보고 제거 전, 후의 샘플의 개수를 비교해봅시다.

In [31]:
print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 1324


샘플의 수가 1,324에서 1,214로 110개의 샘플이 제거되었는데 기존에 출력했던 5개의 샘플을 출력해봅시다.

In [32]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

기존에 네번째, 다섯 번째 샘플에서는 Unknown 값이 있었는데 현재는 제거가 되었습니다. 이제 데이터 전처리를 수행합니다.
여기서 선택한 전처리는 구두점 제거와 단어의 소문자화입니다. 전처리를 수행하고, 다시 샘플 5개를 출력합니다.

In [33]:
def repreprocessing(raw_sentence):
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'unknown',
 'unknown']

기존의 출력과 비교하면 모든 단어들이 소문자화되었으며 N.F.L.이나 Cheerleaders’ 등과 같이 기존에 구두점이 붙어있던 단어들에서 구두점이 제거되었습니다.
단어 집합(vocabulary)을 만들고 크기를 확인합니다.

In [34]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3495


총 3,494개의 단어가 존재합니다. 정수 인코딩을 진행하는 동시에 하나의 문장을 여러 줄로 분해하여 훈련 데이터를 구성합니다.

In [35]:
sequences = list()

for sentence in preprocessed_headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[100, 270],
 [100, 270, 372],
 [100, 270, 372, 1116],
 [100, 270, 372, 1116, 583],
 [100, 270, 372, 1116, 583, 53],
 [100, 270, 372, 1116, 583, 53, 7],
 [100, 270, 372, 1116, 583, 53, 7, 2],
 [100, 270, 372, 1116, 583, 53, 7, 2, 373],
 [100, 270, 372, 1116, 583, 53, 7, 2, 373, 11],
 [100, 270, 372, 1116, 583, 53, 7, 2, 373, 11, 1117],
 [101, 3]]

```
[[99, 269], # former nfl
 [99, 269, 371], # former nfl cheerleaders
 [99, 269, 371, 1115], # former nfl cheerleaders settlement
 [99, 269, 371, 1115, 582], # former nfl cheerleaders settlement offer
 [99, 269, 371, 1115, 582, 52], # 'former nfl cheerleaders settlement offer 1
 [99, 269, 371, 1115, 582, 52, 7], # former nfl cheerleaders settlement offer 1 and
 [99, 269, 371, 1115, 582, 52, 7, 2], # ... 이하 생략 ...
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116], # 모든 단어가 사용된 완전한 첫번째 문장
 # 바로 위의 줄은 : former nfl cheerleaders settlement offer 1 and a meeting with goodell
 [100, 3]] # epa to에 해당되며 두번째 문장이 시작됨.
```

이해를 돕기 위해 출력 결과에 주석을 추가하였습니다. 왜 하나의 문장을 저렇게 나눌까요?
예를 들어 '경마장에 있는 말이 뛰고 있다' 라는 문장 하나가 있을 때, 최종적으로 원하는 훈련 데이터의 형태는 다음과 같습니다.
하나의 단어를 예측하기 위해 이전에 등장한 단어들을 모두 참고하는 것입니다.

![표](img_38.png)

위의 sequences는 모든 문장을 각 단어가 각 시점(time step)마다 하나씩 추가적으로 등장하는 형태로 만들기는 했지만,
아직 예측할 단어에 해당되는 레이블을 분리하는 작업까지는 수행하지 않은 상태입니다.
어떤 정수가 어떤 단어를 의미하는지 알아보기 위해 인덱스로부터 단어를 찾는 index_to_word를 만듭니다.

In [51]:
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))
print('빈도수 상위 583번 단어 : {}'.format(index_to_word[583]))

빈도수 상위 582번 단어 : revolt
빈도수 상위 583번 단어 : offer


582이라는 인덱스를 가진 단어는 본래 offer라는 단어였습니다. 원한다면 다른 숫자로도 시도해보세요.
이제 $y$데이터를 분리하기 전에 전체 샘플의 길이를 동일하게 만드는 패딩 작업을 수행합니다.
패딩 작업을 수행하기 전에 가장 긴 샘플의 길이를 확인합니다.

In [37]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


가장 긴 샘플의 길이인 24로 모든 샘플의 길이를 패딩하겠습니다.

In [38]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0  100  270]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0  100  270  372]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0  100  270  372 1116]]


padding='pre'를 설정하여 샘플의 길이가 24보다 짧은 경우에 앞에 0으로 패딩되었습니다. 이제 맨 우측 단어만 레이블로 분리합니다.

In [40]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [41]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0 100]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0 100 270]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0 100 270 372]]


훈련 데이터 X에서 3개의 샘플만 출력해보았는데, 맨 우측에 있던 정수값 269, 371, 1115가 사라진 것을 볼 수 있습니다.
뿐만 아니라, 각 샘플의 길이가 24에서 23으로 줄었습니다.

In [42]:
print(y[:3])

[ 270  372 1116]


훈련 데이터 y 중 3개의 샘플만 출력해보았는데, 기존 훈련 데이터에서 맨 우측에 있던 정수들이 별도로 저장되었습니다.

In [43]:
y = to_categorical(y, num_classes=vocab_size)

레이블 데이터 y에 대해서 원-핫 인코딩을 수행하였습니다. 이제 모델을 설계합니다.

#### 2) 모델 설계하기

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

하이퍼파라미터인 임베딩 벡터의 차원은 10, 은닉 상태의 크기는 128입니다. 다 대 일 구조의 LSTM을 사용합니다.
전결합층(Fully Connected Layer)을 출력층으로 단어 집합 크기만큼의 뉴런을 배치하여 모델을 설계합니다.
해당 모델은 마지막 시점에서 모든 가능한 단어 중 하나의 단어를 예측하는 다중 클래스 분류 문제를 수행하는 모델입니다.
다중 클래스 분류 문제의 경우, 출력층에 소프트맥스 회귀를 사용해야 하므로 활성화 함수로는 소프트맥스 함수를 사용하고,
손실 함수로 크로스 엔트로피 함수를 사용하여 200 에포크를 수행합니다.

In [45]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)


Epoch 1/200
244/244 - 4s - loss: 7.6388 - accuracy: 0.0256 - 4s/epoch - 18ms/step
Epoch 2/200
244/244 - 2s - loss: 7.1136 - accuracy: 0.0288 - 2s/epoch - 9ms/step
Epoch 3/200
244/244 - 2s - loss: 6.9746 - accuracy: 0.0364 - 2s/epoch - 9ms/step
Epoch 4/200
244/244 - 3s - loss: 6.8402 - accuracy: 0.0424 - 3s/epoch - 12ms/step
Epoch 5/200
244/244 - 2s - loss: 6.6823 - accuracy: 0.0443 - 2s/epoch - 10ms/step
Epoch 6/200
244/244 - 3s - loss: 6.5059 - accuracy: 0.0509 - 3s/epoch - 10ms/step
Epoch 7/200
244/244 - 2s - loss: 6.3179 - accuracy: 0.0519 - 2s/epoch - 10ms/step
Epoch 8/200
244/244 - 2s - loss: 6.1175 - accuracy: 0.0572 - 2s/epoch - 9ms/step
Epoch 9/200
244/244 - 3s - loss: 5.9220 - accuracy: 0.0622 - 3s/epoch - 10ms/step
Epoch 10/200
244/244 - 2s - loss: 5.7365 - accuracy: 0.0672 - 2s/epoch - 9ms/step
Epoch 11/200
244/244 - 3s - loss: 5.5606 - accuracy: 0.0710 - 3s/epoch - 11ms/step
Epoch 12/200
244/244 - 2s - loss: 5.3891 - accuracy: 0.0789 - 2s/epoch - 9ms/step
Epoch 13/200
244/2

문장을 생성하는 함수 sentence_generation을 만들어서 문장을 생성해봅시다.

In [46]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

임의의 단어 'i'에 대해서 10개의 단어를 추가 생성해봅시다.

In [47]:
print(sentence_generation(model, tokenizer, 'i', 10))

i disapprove of school vouchers can i still apply for them


임의의 단어 'how'에 대해서 10개의 단어를 추가 생성해봅시다.

In [48]:
print(sentence_generation(model, tokenizer, 'how', 10))

how to make a crossword puzzle complicated its yet its not


## 08-07 문자 단위 RNN(Char RNN)
***

지금까지 배운 RNN은 전부 입력과 출력의 단위가 단어 벡터였습니다.
하지만 입출력의 단위를 단어 레벨(word-level)에서 문자 레벨(character-level)로 변경하여 RNN을 구현할 수 있습니다.

![그림](img_39.png)

위의 그림은 문자 단위 RNN을 다 대 다(Many-to-Many) 구조로 구현한 경우, 다 대 일(Many-to-One) 구조로 구현한 경우 두 가지를 보여줍니다. 여기서는 이 두 가지 모두 구현해보겠습니다. 첫번째로 구현할 것은 다 대 다 구조를 이용한 언어 모델입니다.

### 1. 문자 단위 RNN 언어 모델(Char RNNLM)
***

이전 시점의 예측 문자를 다음 시점의 입력으로 사용하는 문자 단위 RNN 언어 모델을 구현해봅시다. 앞서 배운 단어 단위 RNN 언어 모델과 다른 점은 단어 단위가 아니라 문자 단위를 입, 출력으로 사용하므로 임베딩층(embedding layer)을 여기서는 사용하지 않겠습니다. 여기서는 언어 모델의 훈련 과정과 테스트 과정의 차이를 이해하는데 집중합니다.

다운로드 링크 : http://www.gutenberg.org/files/11/11-0.txt

고전 소설들은 저작권에 보호받지 않으므로 무료로 다운받을 수 있습니다. 위의 링크에서 '이상한 나라의 앨리스(Alice’s Adventures in Wonderland)'라는 소설을 다운로드 합니다.

#### 1) 데이터에 대한 이해와 전처리

데이터를 로드하고 특수문자를 제거하고 단어를 소문자화하는 간단한 전처리를 수행합니다.

In [126]:
import string
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

# 데이터 로드
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")

f = open('11-0.txt', 'rb')
sentences = []
for sentence in f: # 데이터로부터 한 줄씩 읽는다.
    sentence = sentence.lower() # 소문자화.
    sentence = sentence.decode('ascii', 'ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거
    sentence = sentence.lstrip()
    sentence = sentence.strip() # strip()을 통해 \r, \n을 제거한다.
    if len(sentence) > 0:
        sentences.append(sentence)
f.close()

전처리가 수행된 결과가 리스트에 저장되었습니다. 리스트에서 5개의 원소만 출력해보겠습니다.

In [127]:
sentences[:5]

["*** start of the project gutenberg ebook alice's adventures in",
 'wonderland ***',
 '[illustration]',
 'alices adventures in wonderland',
 'by lewis carroll']

리스트의 원소는 문자열로 구성되어져 있는데 의미있게 문장 토큰화가 된 상태는 아닙니다. 이를 하나의 문자열로 통합하겠습니다.

In [128]:
total_data = ' '.join(sentences)
print('문자열의 길이 또는 총 문자의 개수: %d' % len(total_data))

문자열의 길이 또는 총 문자의 개수: 140323


하나의 문자열로 통합되었고, 문자열의 길이는 약 13만 입니다.(교재 : 약 15만 9천입니다.) 일부 출력해보겠습니다.

In [129]:
print(total_data[:300])

*** start of the project gutenberg ebook alice's adventures in wonderland *** [illustration] alices adventures in wonderland by lewis carroll the millennium fulcrum edition 3.0 contents chapter i.     down the rabbit-hole chapter ii.    the pool of tears chapter iii.   a caucus-race and a long tale 


이 문자열로부터 문자 집합을 만들겠습니다. 기존에는 중복을 제거한 단어들의 모음인 단어 집합(vocabulary)을 만들었으나,
이번에 만들 집합은 단어 집합이 아니라 문자 집합입니다.

In [130]:
char_vocab = sorted(list(set(total_data)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size)) 

문자 집합의 크기 : 43


영어가 훈련 데이터일 때 문자 집합의 크기는 단어 집합을 사용했을 경우보다 집합의 크기가 현저히 작습니다. 아무리 훈련 코퍼스에 수십만 개 이상의 많은 영어 단어가 존재한다고 하더라도, 영어 단어를 표현하기 위해서 사용되는 문자는 26개의 알파벳뿐이기 때문입니다. 만약 훈련 데이터의 알파벳이 대, 소문자가 구분된 상태라고 하더라도 모든 영어 단어는 총 52개의 알파벳으로 표현 가능합니다.

어떤 방대한 양의 텍스트라도 집합의 크기를 적게 가져갈 수 있다는 것은 구현과 테스트를 굉장히 쉽게 할 수 있다는 이점을 가지므로, RNN의 동작 메커니즘 이해를 위한 토이 프로젝트 용도로 유용합니다. 문자 집합의 각 문자에 정수를 부여하고 출력해보겠습니다.

In [131]:
# 문자에 고유한 정수 부여
char_to_index = dict((char, index) for index, char in enumerate(char_vocab))
print('문자 집합 :',char_to_index)

문자 집합 : {' ': 0, '!': 1, "'": 2, '(': 3, ')': 4, '*': 5, ',': 6, '-': 7, '.': 8, '0': 9, '3': 10, ':': 11, ';': 12, '?': 13, '[': 14, ']': 15, '_': 16, 'a': 17, 'b': 18, 'c': 19, 'd': 20, 'e': 21, 'f': 22, 'g': 23, 'h': 24, 'i': 25, 'j': 26, 'k': 27, 'l': 28, 'm': 29, 'n': 30, 'o': 31, 'p': 32, 'q': 33, 'r': 34, 's': 35, 't': 36, 'u': 37, 'v': 38, 'w': 39, 'x': 40, 'y': 41, 'z': 42}


정수 0부터 28(실습16)까지는 공백을 포함한 각종 구두점, 특수문자가 존재하고, 정수 29(실습17)부터 54(실습42)까지는 a부터 z까지 총 26개의 알파벳 소문자가 문자 집합에 포함되어져 있습니다. 반대로 정수로부터 문자를 리턴하는 index_to_char을 만듭니다.

In [132]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

훈련 데이터를 구성해보겠습니다. 훈련 데이터 구성을 위한 간소화 된 예를 들어봅시다. 훈련 데이터에 apple이라는 시퀀스가 있고, 입력의 길이를 4라고 정하였을 때 데이터의 구성은 어떻게 될까요? 입력의 길이가 4이므로 입력 시퀀스와 예측해야 하는 출력 시퀀스 모두 길이는 4가 됩니다. 다시 말해 RNN은 총 네 번의 시점을(timestep)을 가질 수 있다는 의미입니다. apple은 다섯 글자이지만 입력의 길이는 4이므로 'appl'까지만 입력으로 사용할 수 있습니다. 그리고 언어 모델은 다음 시점의 입력을 예측해야하는 모델이므로 'pple'를 예측하도록 데이터가 구성됩니다.

In [135]:
# appl (입력 시퀀스) -> pple (예측해야하는 시퀀스)
# train_X = 'appl'
# train_y = 'pple'

이제 15만 9천의 길이를 가진 문자열로부터 다수의 샘플들을 만들어보겠습니다. 데이터를 만드는 방법은 문장 샘플의 길이를 정하고, 해당 길이만큼 문자열 전체를 등분하는 것입니다. 여기서는 문장의 길이를 60으로 정했는데, 결국 15만 9천을 60으로 나눈 수가 샘플의 수가 됩니다. 몇 개의 샘플을 만들 수 있을지 그 개수를 계산해봅시다.

In [136]:
seq_length = 60

# 문자열의 길이를 seq_length로 나누면 전처리 후 생겨날 샘플 수
n_samples = int(np.floor((len(total_data) - 1) / seq_length))
print ('샘플의 수 : {}'.format(n_samples))

샘플의 수 : 2338


여기서는 총 샘플의 수가 2,658개(실습 : 2338개)가 될 예정입니다. 이제 전처리를 진행합니다. 전처리가 어떻게 진행되었는지는 전처리 후 얻은 train_X와 train_y를 통해 설명하겠습니다.

In [138]:
train_X = []
train_y = []

for i in range(n_samples):
    # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 pick.
    X_sample = total_data[i * seq_length: (i + 1) * seq_length]

    # 정수 인코딩
    X_encoded = [char_to_index[c] for c in X_sample]
    train_X.append(X_encoded)

    # 오른쪽으로 1칸 쉬프트
    y_sample = total_data[i * seq_length + 1: (i + 1) * seq_length + 1]
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

train_X와 train_y의 첫번째 샘플을 출력해봅시다.

In [139]:
print('X 데이터의 첫번째 샘플 :',train_X[0])
print('y 데이터의 첫번째 샘플 :',train_y[0])
print('-'*50)
print('X 데이터의 첫번째 샘플 디코딩 :',[index_to_char[i] for i in train_X[0]])
print('y 데이터의 첫번째 샘플 디코딩 :',[index_to_char[i] for i in train_y[0]])

X 데이터의 첫번째 샘플 : [5, 5, 5, 0, 35, 36, 17, 34, 36, 0, 31, 22, 0, 36, 24, 21, 0, 32, 34, 31, 26, 21, 19, 36, 0, 23, 37, 36, 21, 30, 18, 21, 34, 23, 0, 21, 18, 31, 31, 27, 0, 17, 28, 25, 19, 21, 2, 35, 0, 17, 20, 38, 21, 30, 36, 37, 34, 21, 35, 0]
y 데이터의 첫번째 샘플 : [5, 5, 0, 35, 36, 17, 34, 36, 0, 31, 22, 0, 36, 24, 21, 0, 32, 34, 31, 26, 21, 19, 36, 0, 23, 37, 36, 21, 30, 18, 21, 34, 23, 0, 21, 18, 31, 31, 27, 0, 17, 28, 25, 19, 21, 2, 35, 0, 17, 20, 38, 21, 30, 36, 37, 34, 21, 35, 0, 25]
--------------------------------------------------
X 데이터의 첫번째 샘플 디코딩 : ['*', '*', '*', ' ', 's', 't', 'a', 'r', 't', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'a', 'l', 'i', 'c', 'e', "'", 's', ' ', 'a', 'd', 'v', 'e', 'n', 't', 'u', 'r', 'e', 's', ' ']
y 데이터의 첫번째 샘플 디코딩 : ['*', '*', ' ', 's', 't', 'a', 'r', 't', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' '

train_y[0]은 train_X[0]에서 오른쪽으로 한 칸 쉬프트 된 문장임을 알 수 있습니다.
train_X와 train_y의 두번째 샘플. 즉, 인덱스가 1번인 샘플을 출력하여 데이터의 구성을 확인해봅시다.

In [140]:
print(train_X[1])

[25, 30, 0, 39, 31, 30, 20, 21, 34, 28, 17, 30, 20, 0, 5, 5, 5, 0, 14, 25, 28, 28, 37, 35, 36, 34, 17, 36, 25, 31, 30, 15, 0, 17, 28, 25, 19, 21, 35, 0, 17, 20, 38, 21, 30, 36, 37, 34, 21, 35, 0, 25, 30, 0, 39, 31, 30, 20, 21, 34]


In [141]:
print(train_y[1])

[30, 0, 39, 31, 30, 20, 21, 34, 28, 17, 30, 20, 0, 5, 5, 5, 0, 14, 25, 28, 28, 37, 35, 36, 34, 17, 36, 25, 31, 30, 15, 0, 17, 28, 25, 19, 21, 35, 0, 17, 20, 38, 21, 30, 36, 37, 34, 21, 35, 0, 25, 30, 0, 39, 31, 30, 20, 21, 34, 28]


마찬가지로 train_y[1]은 train_X[1]에서 오른쪽으로 한 칸 쉬프트 된 문장임을 알 수 있습니다. 이제 train_X와 train_y에 대해서 원-핫 인코딩을 수행합니다. 문자 단위 RNN에서는 입력 시퀀스에 대해서 워드 임베딩을 하지 않습니다. 다시 말해 임베딩층(embedding layer)을 사용하지 않을 것이므로, 입력 시퀀스인 train_X에 대해서도 원-핫 인코딩을 합니다.

In [142]:
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)

print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩

train_X의 크기(shape) : (2338, 60, 43)
train_y의 크기(shape) : (2338, 60, 43)


train_X와 train_y의 크기는 2,658 × 60 × 56 입니다. (실습 : 2338, 60, 43)

![그림](img_10.png)

이는 샘플의 수(No. of samples)가 2,658개, 입력 시퀀스의 길이(input_length)가 60, 각 벡터의 차원(input_dim)이 56임을 의미합니다.
원-핫 벡터의 차원은 문자 집합의 크기인 56이어야 하므로 원-핫 인코딩이 수행되었음을 알 수 있습니다.

#### 2) 모델 설계하기

하이퍼파라미터인 은닉 상태의 크기는 256입니다. 모델은 다 대 다 구조의 LSTM을 사용하며, LSTM 은닉층은 두 개를 사용합니다.
전결합층(Fully Connected Layer)을 출력층으로 문자 집합 크기만큼의 뉴런을 배치하여 모델을 설계합니다.
해당 모델은 모든 시점에서 모든 가능한 문자 중 하나의 문자를 예측하는 다중 클래스 분류 문제를 수행하는 모델입니다.
출력층에 소프트맥스 회귀를 사용해야 하므로 활성화 함수로는 소프트맥스 함수를 사용하고, 손실 함수로 크로스 엔트로피 함수를 사용하여 80 에포크를 수행합니다.

In [143]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

hidden_units = 256

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(hidden_units, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=80, verbose=2)

Epoch 1/80
74/74 - 8s - loss: 3.0354 - accuracy: 0.1849 - 8s/epoch - 113ms/step
Epoch 2/80
74/74 - 5s - loss: 2.7142 - accuracy: 0.2515 - 5s/epoch - 74ms/step
Epoch 3/80
74/74 - 5s - loss: 2.3445 - accuracy: 0.3360 - 5s/epoch - 73ms/step
Epoch 4/80
74/74 - 5s - loss: 2.1903 - accuracy: 0.3737 - 5s/epoch - 73ms/step
Epoch 5/80
74/74 - 5s - loss: 2.0860 - accuracy: 0.4015 - 5s/epoch - 73ms/step
Epoch 6/80
74/74 - 5s - loss: 1.9952 - accuracy: 0.4242 - 5s/epoch - 73ms/step
Epoch 7/80
74/74 - 5s - loss: 1.9212 - accuracy: 0.4440 - 5s/epoch - 73ms/step
Epoch 8/80
74/74 - 5s - loss: 1.8604 - accuracy: 0.4603 - 5s/epoch - 73ms/step
Epoch 9/80
74/74 - 5s - loss: 1.7982 - accuracy: 0.4774 - 5s/epoch - 74ms/step
Epoch 10/80
74/74 - 5s - loss: 1.7495 - accuracy: 0.4910 - 5s/epoch - 73ms/step
Epoch 11/80
74/74 - 5s - loss: 1.7033 - accuracy: 0.5024 - 5s/epoch - 73ms/step
Epoch 12/80
74/74 - 5s - loss: 1.6582 - accuracy: 0.5139 - 5s/epoch - 73ms/step
Epoch 13/80
74/74 - 5s - loss: 1.6154 - accuracy

특정 문자를 주면 다음 문자를 계속해서 생성해내는 sentence_generation 함수를 구현합니다. 인자로 학습한 모델. 그리고 모델로 다음 문자를 몇 번 생성할 것인지 횟수를 전달해주면, 해당 함수는 임의로 시작 문자를 정한 뒤에 정해진 횟수만큼의 다음 문자를 지속적으로 예측하여 문장을 생성해냅니다.

In [145]:
def sentence_generation(model, length):
    # 문자에 대한 랜덤한 정수 생성
    ix = [np.random.randint(vocab_size)]

    # 랜덤한 정수로부터 맵핑되는 문자 생성
    y_char = [index_to_char[ix[-1]]]
    print(ix[-1],'번 문자',y_char[-1],'로 예측을 시작!')

    # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성
    X = np.zeros((1, length, vocab_size))

    for i in range(length):
        # X[0][i][예측한 문자의 인덱스] = 1, 즉, 예측 문자를 다음 입력 시퀀스에 추가
        X[0][i][ix[-1]] = 1
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [146]:
result = sentence_generation(model, 100)
print(result)

36 번 문자 t 로 예측을 시작!
1/1 [==============================] - 0s 30ms/step
the gryphon; and then they both sat silent and looked at poor little things of whice was the finst mi


자세히 보면 사실 말이 되지 않는 문장이지만, 언뜻 보기에 그럴듯해 보이는 문장을 생성해냅니다.

### 2. 문자 단위 RNN(Char RNN)으로 텍스트 생성하기
***

이번에는 다 대 일 구조의 RNN을 문자 단위로 학습시키고, 텍스트 생성을 해보겠습니다.

#### 1) 데이터에 대한 이해와 전처리

In [110]:
import numpy as np
from tensorflow.keras.utils import to_categorical

다음과 같이 제가 임의로 만든 엉터리 노래 가사가 있습니다.

In [111]:
raw_text = '''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

위의 텍스트에 존재하는 단락 구분을 없애고 하나의 문자열로 재저장하겠습니다.

In [112]:
tokens = raw_text.split()
raw_text = ' '.join(tokens)
print(raw_text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


단락 구분이 없어지고 하나의 문자열로 재저장되었습니다. 이로부터 문자 집합을 만들어보겠습니다. 기존에는 중복을 제거한 단어들의 모음인 단어 집합(vocabulary)을 만들었으나, 이번에 만들 집합은 단어 집합이 아니라 문자 집합입니다.

In [113]:
# 중복을 제거한 문자 집합 생성
char_vocab = sorted(list(set(raw_text)))
vocab_size = len(char_vocab)
print('문자 집합 :',char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합 : [' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']
문자 집합의 크기 : 33


알파벳 또는 구두점 등의 단위의 집합인 문자 집합이 생성되었습니다. 문자 집합의 크기는 33입니다.

In [114]:
char_to_index = dict((char, index) for index, char in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


이번 실습의 문자 집합의 경우 훈련 데이터에 등장한 알파벳의 대, 소문자를 구분하고 구두점과 공백을 포함하였습니다. 이제 훈련에 사용할 문장 샘플들을 만들어보겠습니다. 여기서는 RNN을 이용한 생성한 텍스트 챕터와 유사하게 데이터를 구성합니다. 다만, 단위가 문자 단위라는 점이 다릅니다. 예를 들어 훈련 데이터에 student라는 단어가 있고, 입력 시퀀스의 길이를 5라고 한다면 입력 시퀀스와 예측해야하는 문자는 다음과 같이 구성됩니다. 5개의 입력 문자 시퀀스로부터 다음 문자 시퀀스를 예측하는 것입니다. 즉, RNN의 시점(timesteps)은 5번입니다.

- stude -> n
- tuden -> t

여기서는 입력 시퀀스의 길이가 10가 되도록 데이터를 구성하겠습니다. 예측 대상인 문자도 필요하므로 길이가 11이 되도록 데이터를 구성합니다.

In [115]:
length = 11
sequences = []
for i in range(length, len(raw_text)):
    seq = raw_text[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

총 훈련 샘플의 수: 426


총 샘플의 수는 426개가 완성되었습니다. 이 중 10개만 출력해보겠습니다.

In [116]:
sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

첫번째 문장이었던 'I get on with life as a programmer,'가 10개의 샘플로 분리된 것을 확인할 수 있습니다. 다른 문장들에 대해서도 sequences에 모두 저장되어져 있습니다. 원한다면, sequences[30:45] 등과 같이 인덱스 범위를 변경하여 출력해보시기 바랍니다. 이제 앞서 만든 char_to_index를 사용하여 전체 데이터에 대해서 정수 인코딩을 수행합니다.

In [117]:
encoded_sequences = []
for sequence in sequences: # 전체 데이터에서 문장 샘플을 1개씩 꺼낸다.
    encoded_sequence = [char_to_index[char] for char in sequence] # 문장 샘플에서 각 문자에 대해서 정수 인코딩을 수행.
    encoded_sequences.append(encoded_sequence)

정수 인코딩 된 결과가 X에 저장되었습니다. 5개만 출력해보겠습니다.

In [118]:
encoded_sequences[:5]

[[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18],
 [0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28],
 [16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17],
 [14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0],
 [28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]]

정수 인코딩이 수행되었습니다. 예측 대상인 문자를 분리시켜주는 작업을 해봅시다. 모든 샘플 문장에 대해서 마지막 문자를 분리하여 마지막 문자가 분리된 샘플은 X_data에 저장하고, 마지막 문자는 y_data에 저장합니다.

In [119]:
encoded_sequences = np.array(encoded_sequences)

# 맨 마지막 위치의 문자를 분리
X_data = encoded_sequences[:,:-1]
# 맨 마지막 위치의 문자를 저장
y_data = encoded_sequences[:,-1]

정상적으로 분리가 되었는지 X와 y 둘 다 5개씩 출력해보겠습니다.

In [120]:
print(X_data[:5])
print(y_data[:5])

[[ 8  0 16 14 28  0 24 23  0 31]
 [ 0 16 14 28  0 24 23  0 31 18]
 [16 14 28  0 24 23  0 31 18 28]
 [14 28  0 24 23  0 31 18 28 17]
 [28  0 24 23  0 31 18 28 17  0]]
[18 28 17  0 21]


앞서 출력한 5개의 샘플에서 각각 맨 뒤의 문자였던 18, 28, 17, 0, 21이 별도로 분리되어 y에 저장되었습니다. 이제 X와 y에 대해서 원-핫 인코딩을 수행해보겠습니다

In [121]:
# 원-핫 인코딩
X_data_one_hot = [to_categorical(encoded, num_classes=vocab_size) for encoded in X_data]
X_data_one_hot = np.array(X_data_one_hot)
y_data_one_hot = to_categorical(y_data, num_classes=vocab_size)

원-핫 인코딩이 수행되었는지 확인하기 위해 수행한 후의 X의 크기(shape)를 보겠습니다.

In [122]:
print(X_data_one_hot.shape)

(426, 10, 33)


현재 X의 크기는 426 × 10 × 33 입니다.

![그림](img_10.png)

이는 샘플의 수(No. of samples)가 426개, 입력 시퀀스의 길이(input_length)가 10, 각 벡터의 차원(input_dim)이 33임을 의미합니다. 원-핫 벡터의 차원은 문자 집합의 크기인 33이어야 하므로 X에 대해서 원-핫 인코딩이 수행되었음을 알 수 있습니다.

#### 2) 모델 설계하기

하이퍼파라미터인 은닉 상태의 크기는 64입니다. 모델은 다 대 일 구조의 LSTM을 사용합니다. 전결합층(Fully Connected Layer)을 출력층으로 문자 집합 크기만큼의 뉴런을 배치하여 모델을 설계합니다. 해당 모델은 마지막 시점에서 모든 가능한 문자 중 하나의 문자를 예측하는 다중 클래스 분류 문제를 수행하는 모델입니다. 다중 클래스 분류 문제의 경우, 출력층에 소프트맥스 회귀를 사용해야 하므로 활성화 함수로는 소프트맥스 함수를 사용하고, 손실 함수로 크로스 엔트로피 함수를 사용하여 100 에포크를 수행합니다.

In [123]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

hidden_units = 64

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(X_data_one_hot.shape[1], X_data_one_hot.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_data_one_hot, y_data_one_hot, epochs=100, verbose=2)

Epoch 1/100
14/14 - 2s - loss: 3.4753 - accuracy: 0.1033 - 2s/epoch - 115ms/step
Epoch 2/100
14/14 - 0s - loss: 3.3786 - accuracy: 0.1972 - 40ms/epoch - 3ms/step
Epoch 3/100
14/14 - 0s - loss: 3.1320 - accuracy: 0.1972 - 44ms/epoch - 3ms/step
Epoch 4/100
14/14 - 0s - loss: 3.0058 - accuracy: 0.1972 - 43ms/epoch - 3ms/step
Epoch 5/100
14/14 - 0s - loss: 2.9687 - accuracy: 0.1972 - 42ms/epoch - 3ms/step
Epoch 6/100
14/14 - 0s - loss: 2.9548 - accuracy: 0.1972 - 42ms/epoch - 3ms/step
Epoch 7/100
14/14 - 0s - loss: 2.9415 - accuracy: 0.1972 - 42ms/epoch - 3ms/step
Epoch 8/100
14/14 - 0s - loss: 2.9226 - accuracy: 0.1972 - 42ms/epoch - 3ms/step
Epoch 9/100
14/14 - 0s - loss: 2.9100 - accuracy: 0.1972 - 42ms/epoch - 3ms/step
Epoch 10/100
14/14 - 0s - loss: 2.8958 - accuracy: 0.1972 - 41ms/epoch - 3ms/step
Epoch 11/100
14/14 - 0s - loss: 2.8858 - accuracy: 0.1972 - 42ms/epoch - 3ms/step
Epoch 12/100
14/14 - 0s - loss: 2.8641 - accuracy: 0.1972 - 41ms/epoch - 3ms/step
Epoch 13/100
14/14 - 0s -

문장을 생성하는 함수 sentence_generation을 만들어서 문장을 생성해봅시다.
해당 함수는 문자열을 입력하면, 해당 문자열로부터 다음 문자를 예측하는 것을 반복하여 최종적으로 문장을 완성합니다.

In [124]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):

    # 초기 시퀀스
    init_text = seed_text
    sentence = ''

    # 다음 문자 예측은 총 n번만 반복.
    for _ in range(n):
        encoded = [char_to_index[char] for char in seed_text] # 현재 시퀀스에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre') # 데이터에 대한 패딩
        encoded = to_categorical(encoded, num_classes=len(char_to_index))

        # 입력한 X(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 문자)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for char, index in char_to_index.items():
            if index == result:
                break

        # 현재 시퀀스 + 예측 문자를 현재 시퀀스로 변경
        seed_text = seed_text + char

        # 예측 문자를 문장에 저장
        sentence = sentence + char

    # n번의 다음 문자 예측이 끝나면 최종 완성된 문장을 리턴.
    sentence = init_text + sentence
    return sentence

In [125]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

I get on with life as a programmer, I like to con emplate aner. Mt I hnnftt llte t, e e t 


두 개의 문장이 출력되었습니다. 이 두 문장은 훈련 데이터에서는 연속적으로 나온 적이 없는 두 문장임에도 모델이 임의로 생성해냈습니다.